<a href="https://colab.research.google.com/github/kchamarty/pythonproject/blob/master/Copy_of_retail_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import pandas_profiling

#%matplotlib notebook
import matplotlib as plot
data_retail = pd.read_csv('https://raw.githubusercontent.com/kchamarty/pythonproject/master/data/retail_train.csv')
print(data_retail.columns)

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')


In [0]:
from scipy.stats import mode

outlet_size_mode =data_retail.pivot_table(values ='Outlet_Size',columns='Outlet_Type',aggfunc=(lambda x:mode(x).mode[0]))
print(outlet_size_mode)
#create boolean matrix for null values 
miss_bool =data_retail.Outlet_Size.isnull()

print('Original Missing Values:',sum(miss_bool))

#replace missing Outlet_Size values by it's mode grouped by Outlet_Type 

data_retail.loc[miss_bool,'Outlet_Size'] = data_retail.loc[miss_bool,'Outlet_Type'].apply(lambda x:outlet_size_mode[x])

miss_bool =data_retail.Outlet_Size.isnull()
print('After substituting Modes:',sum(miss_bool))

Outlet_Type Grocery Store Supermarket Type1 Supermarket Type2 Supermarket Type3
Outlet_Size         Small             Small            Medium            Medium
Original Missing Values: 2410
After substituting Modes: 0


In [0]:
# combine the Item_Fat_content of low Fat, LF & low fat  into "low fat and Regular and reg into Regular

print(data_retail.Item_Fat_Content.value_counts())

data_retail.Item_Fat_Content.replace({'LF':'low fat', 'Low Fat':'low fat','reg':'Regular'},inplace=True)
print(data_retail.Item_Fat_Content.value_counts())


Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64
low fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64


In [0]:
print(data_retail.head(5))

  Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility  \
0           FDA15         9.30          low fat         0.016047   
1           DRC01         5.92          Regular         0.019278   
2           FDN15        17.50          low fat         0.016760   
3           FDX07        19.20          Regular         0.000000   
4           NCD19         8.93          low fat         0.000000   

               Item_Type  Item_MRP Outlet_Identifier  \
0                  Dairy  249.8092            OUT049   
1            Soft Drinks   48.2692            OUT018   
2                   Meat  141.6180            OUT049   
3  Fruits and Vegetables  182.0950            OUT010   
4              Household   53.8614            OUT013   

   Outlet_Establishment_Year Outlet_Size Outlet_Location_Type  \
0                       1999      Medium               Tier 1   
1                       2009      Medium               Tier 3   
2                       1999      Medium               Tier

In [0]:
#part of feature engineering

print(data_retail.Item_Identifier.str.slice(0,2).value_counts())
values={'FD':'food','NC':'non-consumables','DR':'drinks'}
print(values);
data_retail['code_exp'] = data_retail.Item_Identifier.apply(lambda  x : values[x[0:2]])

print(data_retail.code_exp.value_counts())

FD    6125
NC    1599
DR     799
Name: Item_Identifier, dtype: int64
{'FD': 'food', 'NC': 'non-consumables', 'DR': 'drinks'}
food               6125
non-consumables    1599
drinks              799
Name: code_exp, dtype: int64


In [0]:
#should we combine outlets supermarket Type2 and Type3

print(data_retail.pivot_table(values='Item_Outlet_Sales',index ='Outlet_Type'))

                   Item_Outlet_Sales
Outlet_Type                         
Grocery Store             339.828500
Supermarket Type1        2316.181148
Supermarket Type2        1995.498739
Supermarket Type3        3694.038558


In [0]:
#Age of store in years:
from datetime import date
#curryear=date.today()
#print(curryear.year)
#data_retail.Outlet_Establishment_Year[0])

data_retail['Outlet_age']=2013-data_retail.Outlet_Establishment_Year
print(data_retail['Outlet_age'].describe())

count    8523.000000
mean       15.168133
std         8.371760
min         4.000000
25%         9.000000
50%        14.000000
75%        26.000000
max        28.000000
Name: Outlet_age, dtype: float64


In [0]:
#Encoding

#Label Encoding - adds weight to the x
#one Hot Encoding - 